# Dependencies

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
import json
from sqlalchemy import create_engine, inspect
import numpy as np
import time
import datetime
from config import *
import psycopg2
from psycopg2 import Error
#Database connection definitions in config

# Database Connection

In [2]:
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

### View Table Names in Database

In [3]:
inspector=inspect(engine)
inspector.get_table_names()

['category', 'headlines', 'sources']

In [4]:
for column in inspector.get_columns("sources"):
    print(column)

{'name': 'source_id', 'type': INTEGER(), 'nullable': False, 'default': "nextval('sources_source_id_seq'::regclass)", 'autoincrement': True, 'comment': None}
{'name': 'source_name', 'type': VARCHAR(length=100), 'nullable': False, 'default': None, 'autoincrement': False, 'comment': None}
{'name': 'last_updated', 'type': TIMESTAMP(), 'nullable': False, 'default': 'LOCALTIMESTAMP', 'autoincrement': False, 'comment': None}


# Web Scrape

In [5]:
# URL of page to be scraped
url = 'https://www.allsides.com/unbiased-balanced-news'
response = requests.get(url)
response.status_code

200

In [6]:
html = response.content
soup = bs(html, "lxml")

# Sources 
source = soup.find("div", class_="row-fluid bias-trio-wrapper")
source_name = source.find_all("div", class_="news-source")

# Articles
article = soup.find("div", class_="row-fluid bias-trio-wrapper")
article_name = article.find_all("div", class_="news-title")

# Category
category_name = soup.find_all("div", class_="news-topic")

# Sub Header
article_sub_header = soup.find_all("div", class_="topic-description")

# Source url
article_url = soup.find("div", class_="row-fluid bias-trio-wrapper")
article_urls = article_url.find_all('div', class_="news-title")

In [7]:
print(len(article_urls))
print(len(article_sub_header))
print(len(category_name))
print(len(article_name))
print(len(source_name))

45
45
45
45
45


### Append Scraped Data to Lists

In [8]:
sources = []
articles = []
categories = []
sub_headers = []
urls = []

for x in source_name:
    source = x.text.strip()
    sources.append(source)
    
for x in article_name:
    article = x.text.strip()
    articles.append(article)
    
for x in category_name:
    category = x.text.strip()
    categories.append(category)

for x in article_sub_header:
    sub_header = x.text.strip()
    sub_headers.append(sub_header)

for x in article_urls:
    urls.append(x.a.get('href'))

### Unique Categories/Sources

In [9]:
categories_unique = []
for x in categories:
    if x not in categories_unique:
        categories_unique.append(x)
print(len(categories_unique))

sources_unique = []
for x in sources:
    if x not in sources_unique:
        sources_unique.append(x)
print(len(sources_unique))

28
34


In [10]:
sources_df = pd.DataFrame()
sources_df['source_name'] = sources_unique
sources_df.tail()

,source_name
29,Breitbart News
30,Reason
31,Washington Examiner
32,The Epoch Times
33,The Dispatch


In [11]:
category_df = pd.DataFrame()
category_df['category_name'] = categories_unique
category_df.tail()

,category_name
23,White House
24,Arts and Entertainment
25,Kamala Harris
26,Energy
27,COVID-19 Misinformation


In [12]:
headlines_df = pd.DataFrame()

headlines_df['article_name'] = articles
headlines_df['article_sub_header'] = sub_headers
headlines_df['source_name'] = sources
headlines_df['category_name'] = categories
headlines_df['url'] = urls

headlines_df.tail()

,article_name,article_sub_header,source_name,category_name,url
40,Portland mayor: Time to ‘take our city back’ f...,"he mayor of Portland, Ore. has asked for the p...",New York Post (News),Violence in America,https://www.allsides.com/news/2021-04-25-1037/...
41,Portland police declare riot Friday night afte...,"Police in Portland, Oregon, declared a riot in...",Fox News (Online News),Violence in America,https://www.allsides.com/news/2021-04-24-1757/...
42,Kamala Harris to have virtual conversation wit...,Vice President Kamala Harris and Mexican Presi...,Washington Examiner,Kamala Harris,https://www.allsides.com/news/2021-04-24-1748/...
43,California to Halt Fracking Permits by January...,California will stop issuing new permits for h...,The Epoch Times,Energy,https://www.allsides.com/news/2021-04-25-0152/...
44,Could the mRNA Vaccines Permanently Alter DNA?,"FACT CHECK An article from The Defender, a new...",The Dispatch,COVID-19 Misinformation,https://www.allsides.com/news/2021-04-25-0145/...


# Uploading to SQL DB

### Sources Table

In [13]:
conn = engine.connect()
sources_df.to_sql("sources", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

In [14]:
query = """
        SELECT
            *
        FROM
            sources
        """

conn = engine.connect()
sql_sources_df = pd.read_sql(query, con=conn)
conn.close()

sql_sources_df.tail()

,source_id,source_name,last_updated
29,30,Breitbart News,2021-04-26 13:31:39.564717
30,31,Reason,2021-04-26 13:31:39.564717
31,32,Washington Examiner,2021-04-26 13:31:39.564717
32,33,The Epoch Times,2021-04-26 13:31:39.564717
33,34,The Dispatch,2021-04-26 13:31:39.564717


### Category Table

In [15]:
conn = engine.connect()
category_df.to_sql("category", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

In [16]:
query = """
        SELECT
            *
        FROM
            category
        """
#Save to Df
conn = engine.connect()
sql_cat_df = pd.read_sql(query, con=conn)
conn.close()

sql_cat_df.tail()

,category_id,category_name,last_updated
23,24,White House,2021-04-26 13:31:39.608942
24,25,Arts and Entertainment,2021-04-26 13:31:39.608942
25,26,Kamala Harris,2021-04-26 13:31:39.608942
26,27,Energy,2021-04-26 13:31:39.608942
27,28,COVID-19 Misinformation,2021-04-26 13:31:39.608942


### Headlines Table

In [17]:
source_ids = []
category_ids = []

for x in headlines_df.source_name:
    source_ids.append(sql_sources_df.source_id.loc[sql_sources_df['source_name'] == x].values[0])
    
for x in headlines_df.category_name:
    category_ids.append(sql_cat_df.category_id.loc[sql_cat_df['category_name'] == x].values[0])

In [18]:
sql_headlines_df = pd.DataFrame()

sql_headlines_df['article_name'] = articles
sql_headlines_df['article_sub_header'] = sub_headers
sql_headlines_df['source_id'] = source_ids
sql_headlines_df['category_id'] = category_ids
sql_headlines_df['url'] = urls

sql_headlines_df.tail()

,article_name,article_sub_header,source_id,category_id,url
40,Portland mayor: Time to ‘take our city back’ f...,"he mayor of Portland, Ore. has asked for the p...",24,20,https://www.allsides.com/news/2021-04-25-1037/...
41,Portland police declare riot Friday night afte...,"Police in Portland, Oregon, declared a riot in...",28,20,https://www.allsides.com/news/2021-04-24-1757/...
42,Kamala Harris to have virtual conversation wit...,Vice President Kamala Harris and Mexican Presi...,32,26,https://www.allsides.com/news/2021-04-24-1748/...
43,California to Halt Fracking Permits by January...,California will stop issuing new permits for h...,33,27,https://www.allsides.com/news/2021-04-25-0152/...
44,Could the mRNA Vaccines Permanently Alter DNA?,"FACT CHECK An article from The Defender, a new...",34,28,https://www.allsides.com/news/2021-04-25-0145/...


In [19]:
conn = engine.connect()
sql_headlines_df.to_sql("headlines", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

In [20]:
query = """
        SELECT
            *
        FROM
            headlines
        """
#Save to Df
conn = engine.connect()
sql_headlines_df = pd.read_sql(query, con=conn)
conn.close()

sql_headlines_df.tail()

,article_id,article_name,article_sub_header,source_id,category_id,url,last_updated
40,41,Portland mayor: Time to ‘take our city back’ f...,"he mayor of Portland, Ore. has asked for the p...",24,20,https://www.allsides.com/news/2021-04-25-1037/...,2021-04-26 13:31:39.689265
41,42,Portland police declare riot Friday night afte...,"Police in Portland, Oregon, declared a riot in...",28,20,https://www.allsides.com/news/2021-04-24-1757/...,2021-04-26 13:31:39.689265
42,43,Kamala Harris to have virtual conversation wit...,Vice President Kamala Harris and Mexican Presi...,32,26,https://www.allsides.com/news/2021-04-24-1748/...,2021-04-26 13:31:39.689265
43,44,California to Halt Fracking Permits by January...,California will stop issuing new permits for h...,33,27,https://www.allsides.com/news/2021-04-25-0152/...,2021-04-26 13:31:39.689265
44,45,Could the mRNA Vaccines Permanently Alter DNA?,"FACT CHECK An article from The Defender, a new...",34,28,https://www.allsides.com/news/2021-04-25-0145/...,2021-04-26 13:31:39.689265


# Query to the Database in Postgres

In [21]:
# Find all articles related to CORONA VIRUS. Sort them by the SOURCE in alphabetical order.
query = """
        SELECT
            h.article_id,
            h.article_name,
            s.source_name,
            c.category_name
        FROM
            headlines h
        JOIN
            category c ON 
            h.category_id = c.category_id
        JOIN 
            sources s ON
            h.source_id = s.source_id
        WHERE
            c.category_id = 5
        ORDER BY
            s.source_name asc;
        """

#Save to Df
conn = engine.connect()
query_df = pd.read_sql(query, con=conn)
conn.close()

query_df.tail()

,article_id,article_name,source_name,category_name
0,5,Liberal push to expand Supreme Court is all bu...,Politico,US Senate
1,21,GOP worries fiscal conservatism losing its ral...,The Hill,US Senate


# Updating the Database

### Sources Table

In [22]:
# Pull sources Table from SQL db
query = """
        SELECT
            *
        FROM
            sources
        """
conn = engine.connect()
sql_sources_df = pd.read_sql(query, con=conn)
conn.close()

sql_sources_df.tail()

,source_id,source_name,last_updated
29,30,Breitbart News,2021-04-26 13:31:39.564717
30,31,Reason,2021-04-26 13:31:39.564717
31,32,Washington Examiner,2021-04-26 13:31:39.564717
32,33,The Epoch Times,2021-04-26 13:31:39.564717
33,34,The Dispatch,2021-04-26 13:31:39.564717


In [23]:
# Pull current list of Source (we are avoiding duplicating the list here)
sql_sources_list = sql_sources_df.source_name.to_list()

# Create empty list for updates
sources_update = []

# Loop through current list of Sources
for source in sources_df.source_name:
    # If source is NOT IN current list... Append to our update list
    if source not in sql_sources_list:
        sources_update.append(source)

source_adds = len(sources_update)
print(f"The number of sources to be added to SQL is: {source_adds}")

# Save updates in a new DataFrame
sources_update_df = pd.DataFrame()
sources_update_df['source_name'] = sources_update
sources_update_df.head()

The number of sources to be added to SQL is: 0


,source_name


In [24]:
# Send UPDATES DataFrame to SQL
# Any NEW sources will be added to the end of the table
conn = engine.connect()
sources_update_df.to_sql("sources", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

### Category Table

In [25]:
query = """
        SELECT
            *
        FROM
            category
        """
conn = engine.connect()
sql_cat_df = pd.read_sql(query, con=conn)
conn.close()

sql_cat_df.tail()

,category_id,category_name,last_updated
23,24,White House,2021-04-26 13:31:39.608942
24,25,Arts and Entertainment,2021-04-26 13:31:39.608942
25,26,Kamala Harris,2021-04-26 13:31:39.608942
26,27,Energy,2021-04-26 13:31:39.608942
27,28,COVID-19 Misinformation,2021-04-26 13:31:39.608942


In [26]:
# Pull current list of Categories (we are avoiding duplicating the list here)
sql_cat_list = sql_cat_df.category_name.to_list()

# Create empty list for updates
cats_update = []

# Loop through current list of Categories
for cat in category_df.category_name:
    # If category is NOT IN current list... Append to our update list
    if cat not in sql_cat_list:
        cats_update.append(cat)

cat_adds = len(cats_update)
print(f"The number of sources to be added to SQL is: {cat_adds}")

# Save updates in a new DataFrame
cat_update_df = pd.DataFrame()
cat_update_df['category_name'] = cats_update
cat_update_df.head()

The number of sources to be added to SQL is: 0


,category_name


In [27]:
conn = engine.connect()
cat_update_df.to_sql("category", schema="public", con=conn, if_exists="append", index=False, method="multi")
conn.close()

### Headlines Table

In [28]:
query = """
        SELECT
            *
        FROM
            headlines
        """
conn = engine.connect()
sql_headlines_df = pd.read_sql(query, con=conn)
conn.close()

sql_headlines_df.tail()

,article_id,article_name,article_sub_header,source_id,category_id,url,last_updated
40,41,Portland mayor: Time to ‘take our city back’ f...,"he mayor of Portland, Ore. has asked for the p...",24,20,https://www.allsides.com/news/2021-04-25-1037/...,2021-04-26 13:31:39.689265
41,42,Portland police declare riot Friday night afte...,"Police in Portland, Oregon, declared a riot in...",28,20,https://www.allsides.com/news/2021-04-24-1757/...,2021-04-26 13:31:39.689265
42,43,Kamala Harris to have virtual conversation wit...,Vice President Kamala Harris and Mexican Presi...,32,26,https://www.allsides.com/news/2021-04-24-1748/...,2021-04-26 13:31:39.689265
43,44,California to Halt Fracking Permits by January...,California will stop issuing new permits for h...,33,27,https://www.allsides.com/news/2021-04-25-0152/...,2021-04-26 13:31:39.689265
44,45,Could the mRNA Vaccines Permanently Alter DNA?,"FACT CHECK An article from The Defender, a new...",34,28,https://www.allsides.com/news/2021-04-25-0145/...,2021-04-26 13:31:39.689265


## Update Headlines

In [29]:
sql_headlines_list = sql_headlines_df.article_name.to_list()

headlines_update = []
count = 0

for article in headlines_df.article_name:
    if article not in sql_headlines_list:
        headlines_update.append(article)
        if len(headlines_update) > 0:
            #Find the article
            update_df = headlines_df.loc[headlines_df.article_name == article]
            headline_updater = update_df.article_name.values[0]
            sub_header_updater = update_df.article_sub_header.values[0]
            source_id_updater = sql_sources_df.source_id.loc[sql_sources_df['source_name'] == update_df.source_name.values[0]]
            category_id_updater = sql_cat_df.category_id.loc[sql_cat_df['category_name'] == update_df.category_name.values[0]]
            url_updater = update_df.url.values[0]
            
             #Save variables off to DataFrame
            updater_df = pd.DataFrame()

            updater_df['article_name'] = update_df.article_name
            updater_df['article_sub_header'] = sub_header_updater
            updater_df['source_id'] = source_id_updater.values[0]
            updater_df['category_id'] = category_id_updater.values[0]
            updater_df['url'] = url_updater
            
               #Load to SQL
            conn = engine.connect()
            updater_df.to_sql("headlines", schema="public", con=conn, 
                                          if_exists="append", index=False, method="multi")
            conn.close()

            print(f"NOT FOUND - Added {article} to SQL headlines table")
            count = count + 1
            headlines_update = []
    else: 
        count = count + 1
        print(f"ARTICLE FOUND - Skipping headline: {count} of {len(headlines_df.article_name)}")
            
        

ARTICLE FOUND - Skipping headline: 1 of 45
ARTICLE FOUND - Skipping headline: 2 of 45
ARTICLE FOUND - Skipping headline: 3 of 45
ARTICLE FOUND - Skipping headline: 4 of 45
ARTICLE FOUND - Skipping headline: 5 of 45
ARTICLE FOUND - Skipping headline: 6 of 45
ARTICLE FOUND - Skipping headline: 7 of 45
ARTICLE FOUND - Skipping headline: 8 of 45
ARTICLE FOUND - Skipping headline: 9 of 45
ARTICLE FOUND - Skipping headline: 10 of 45
ARTICLE FOUND - Skipping headline: 11 of 45
ARTICLE FOUND - Skipping headline: 12 of 45
ARTICLE FOUND - Skipping headline: 13 of 45
ARTICLE FOUND - Skipping headline: 14 of 45
ARTICLE FOUND - Skipping headline: 15 of 45
ARTICLE FOUND - Skipping headline: 16 of 45
ARTICLE FOUND - Skipping headline: 17 of 45
ARTICLE FOUND - Skipping headline: 18 of 45
ARTICLE FOUND - Skipping headline: 19 of 45
ARTICLE FOUND - Skipping headline: 20 of 45
ARTICLE FOUND - Skipping headline: 21 of 45
ARTICLE FOUND - Skipping headline: 22 of 45
ARTICLE FOUND - Skipping headline: 23 of 